In [1]:
import matplotlib.pyplot as plt
import torch
%matplotlib auto

Using matplotlib backend: Qt5Agg


In [2]:
def multibox_prior(data,sizes,ratios):
    in_height,in_weight=data.shape[-2:]
    num_sizes,num_ratios=len(sizes),len(ratios)
    boxes_per_pixel=(num_ratios+num_sizes-1)
    size_tensor=torch.tensor(sizes)
    ratios_tensor=torch.tensor(ratios)
    
    offset_h,offset_w=0.5,0.5
    step_h,step_w=1.0/in_height,1.0/in_weight
    
    center_h=(torch.arange(in_height)+offset_h)*step_h
    center_w=(torch.arange(in_weight)+offset_w)*step_w
    shift_y,shift_x=torch.meshgrid(center_h,center_w)
    shift_y,shift_x=shift_y.reshape(-1),shift_x.reshape(-1)
    
    w=torch.cat((size_tensor*torch.sqrt(ratios_tensor[0]),size_tensor[0]*torch.sqrt(ratios_tensor[1:])))*in_height/in_weight
    h=torch.cat((size_tensor/torch.sqrt(ratios_tensor[0]),size_tensor[0]/torch.sqrt(ratios_tensor[1:])))
    
    anchor_manipulation=torch.stack((-w,-h,w,h)).T.repeat(in_height*in_weight,1)/2
    
    out_grid=torch.stack([shift_x,shift_y,shift_x,shift_y],dim=1).repeat_interleave(boxes_per_pixel,dim=0)
    output=out_grid+anchor_manipulation
    return output.unsqueeze(0)
    
    

In [3]:
img=plt.imread(r'F:\study\ml\LM\image\13\catdog.jpg')
h,w=img.shape[:2]
print(h,w)

561 728


In [4]:
X=torch.rand(size=(1,3,h,w))
Y=multibox_prior(X,sizes=[0.75,0.5,0.25],ratios=[1,2,0.5])
Y.shape

D:\ProgramData\Anaconda3\lib\site-packages\torch\functional.py:568: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\TensorShape.cpp:2228.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


torch.Size([1, 2042040, 4])

In [5]:
boxes = Y.reshape(h, w, 5, 4)

In [6]:
boxes.shape

torch.Size([561, 728, 5, 4])

In [7]:
def bbox_to_rect(bbox,color):
    return plt.Rectangle(xy=(bbox[0],bbox[1]),width=bbox[2]-bbox[0],height=bbox[3]-bbox[1],fill=False,edgecolor=color,linewidth=2)

In [8]:
bbox_to_rect(bbox.detach().numpy(),color)

NameError: name 'bbox' is not defined

In [ ]:
bbox_scale = torch.tensor((w, h, w, h))
# box1=(boxes[250, 250, 2, :]*bbox_scale).detach().numpy()
box1=(boxes[250, 250, :, :]*bbox_scale).detach().numpy()
plt.imshow(img)
for i in range(box1.shape[-2]):
    rect=plt.Rectangle(xy=(box1[i][0],box1[i][1]),width=box1[i][2]-box1[i][0],height=box1[i][3]-box1[i][1],fill=False,edgecolor='r',linewidth=2)
# print(rect)
    plt.gca().add_patch(rect)
plt.show()

In [9]:
box1.shape

NameError: name 'box1' is not defined

In [10]:
def show_bboxes(axes,bboxes,labels=None,colors=None):
    def _make_list(obj,default_values=None):
        if obj is None:
            obj=default_values
        elif not isinstance(obj,(list,tuple)):
            obj=[obj]
        return obj
    labels=_make_list(labels)
    colors=_make_list(colors,['b','g','r','m','c'])
    plt.imshow(axes)
    for i ,bbox in enumerate(bboxes):
        color=colors[i % len(colors)]
        rect=bbox_to_rect(bbox.detach().numpy(),color)
        plt.gca().add_patch(rect)
        if labels and len(labels)>i:
            text_color='k' if color=='w' else 'w'
            plt.text(rect.xy[0],rect.xy[1],labels[i],va='center',ha='center',fontsize=9,color=text_color,bbox=dict(facecolor=color,lw=0))
    plt.show()

In [11]:
h,w=img.shape[:2]

bbox_scale = torch.tensor((w, h, w, h))
show_bboxes(img, boxes[250, 250, :, :] * bbox_scale,['s=0.75, r=1', 's=0.5, r=1', 's=0.25, r=1', 's=0.75, r=2','s=0.75, r=0.5'])

In [12]:
def box_iou(boxes1,boxes2):
    box_area=lambda boxes:((boxes[:,2]-boxes[:,0])*(boxes[:,3]-boxes[:,1]))
    areas1=box_area(boxes1)
    areas2=box_area(boxes2)
    inter_upperleft=torch.max(boxes1[:,None,:2],boxes2[:,:2])
    inter_lowerright=torch.min(boxes1[:,None,2:],boxes2[:,2:])
    inters=(inter_lowerright - inter_upperleft).clamp(min=0)
    inter_areas=inters[:,:,0] * inter[:,:,1]
    union_areas=areas1[:,None]+areas2-inter_areas
    return inter_areas/union_areas
    

In [86]:
boxes.shape

torch.Size([561, 728, 5, 4])

In [96]:
box2=boxes[0]
box2.shape

torch.Size([728, 5, 4])

In [98]:
box2[:,None,:2].shape

torch.Size([728, 1, 2, 4])

In [99]:
box2[:,:2].shape

torch.Size([728, 2, 4])

In [100]:
torch.max(box2[:,None,:2],box2[:,:2]).shape

torch.Size([728, 728, 2, 4])

In [85]:
torch.max(torch.Tensor([[1,2,3],[4,5,6]]),torch.Tensor([[3,2,1],[4,4,4]]))

tensor([[3., 2., 3.],
        [4., 5., 6.]])

box_iou 分解

In [41]:
def box_iou(boxes1,boxes2):
    box_area=lambda boxes:((boxes[:,2]-boxes[:,0])*(boxes[:,3]-boxes[:,1]))
    areas1=box_area(boxes1)
    areas2=box_area(boxes2)
    inter_upperleft=torch.max(boxes1[:,None,:2],boxes2[:,:2])
    inter_lowerright=torch.min(boxes1[:,None,2:],boxes2[:,2:])
    inters=(inter_lowerright - inter_upperleft).clamp(min=0)
    inter_areas=inters[:,:,0] * inters[:,:,1]
    union_areas=areas1[:,None]+areas2-inter_areas
    return inter_areas/union_areas

In [13]:
ground_truth = torch.tensor([[0, 0.1, 0.08, 0.52, 0.92],[1, 0.55, 0.2, 0.9, 0.88]])
anchors = torch.tensor([[0, 0.1, 0.2, 0.3], [0.15, 0.2, 0.4, 0.4],[0.63, 0.05, 0.88, 0.98], [0.66, 0.45, 0.8, 0.8],[0.57, 0.3, 0.92, 0.9]])

In [20]:
ground_truth[1][1:].unsqueeze(dim=0)

tensor([[0.5500, 0.2000, 0.9000, 0.8800]])

In [19]:
anchors[0].unsqueeze(dim=0)

tensor([[0.0000, 0.1000, 0.2000, 0.3000]])

In [21]:
box_area=lambda boxes:((boxes[:,2]-boxes[:,0])*(boxes[:,3]-boxes[:,1]))
box_area(anchors[0].unsqueeze(dim=0))

tensor([0.0400])

In [22]:
box_area(ground_truth[1][1:].unsqueeze(dim=0))

tensor([0.2380])

In [23]:
box11=anchors[0].unsqueeze(dim=0)
box22=ground_truth[1][1:].unsqueeze(dim=0)

In [27]:
box22[:,:2]

tensor([[0.5500, 0.2000]])

In [26]:
box11

tensor([[0.0000, 0.1000, 0.2000, 0.3000]])

In [25]:
box11[:,None,:2]

tensor([[[0.0000, 0.1000]]])

In [28]:
torch.max(box11[:,None,:2],box22[:,:2])

tensor([[[0.5500, 0.2000]]])

In [29]:
torch.max(box11[:,None,:2],box22[:,:2]).shape

torch.Size([1, 1, 2])

In [ ]:
torch.max(box11[:,None,:2],box22[:,:2])

In [34]:
box_area(box22)

tensor([0.2380])

In [32]:
box_area(box11)[:,None]

tensor([[0.0400]])

In [35]:
box_area(box11)[:,None]+box_area(box22)

tensor([[0.2780]])

In [51]:
def box_iou(boxes1,boxes2):
    box_area=lambda boxes : ((boxes[:,2]-boxes[:,0])*(boxes[:,3]-boxes[:,1]))
    areas1=box_area(boxes1)
    areas2=box_area(boxes2)
    
    inter_upperleft=torch.max(boxes1[:,None,:2],boxes2[:,:2])
    inter_lowerright=torch.min(boxes1[:,None,2:],boxes2[:,2:])
    inters=(inter_lowerright-inter_upperleft).clamp(min=0)
    print('inters :',inters )
    inter_areas=inters[:,:,0]*inters[:,:,1]
    print('inter_areas: ',inter_areas)
    union_areas=areas1[:,None]+areas2-inter_areas
    
    return inter_areas/union_areas
    

In [42]:
def assign_anchor_to_bbox(groud_truth,anchors,device,iou_threshold=0.5):
    num_anchors,num_gt_boxes=anchors.shape[0],groud_truth.shape[0]
    jaccard=box_iou(anchors,groud_truth)
    anchors_bbox_map=torch.full((num_anchors),-1,dtpe=torch.long,device=device)
    max_ious,indices=torch.max(jaccard,dim=1)
    anc_i=torch.nonzero(max_ious>=0.5).reshape(-1)
    box_j=indices[max_ious>=0.5]
    anchors_bbox_map[anc_i]=box_j
    col_discard=torch.full((num_anchors,),-1)
    row_discard=torch.full((num_gt_boxes,),-1)
    
    for _ in range(num_gt_boxes):
        max_idx=torch.argmax(jaccard)
        box_idx=(max_idx % num_gt_boxes).long()
        anc_idx=(max_idx / num_get_boxes).long()
        anchors_bbox_map[anc_idx]=box_idx
        jaccard[:,box_idx]=col_discard
        jaccard[anc_idx,:]=row_discard
        
    return anchors_bbox_map    

In [52]:
ground_truth = torch.tensor([[0, 0.1, 0.08, 0.52, 0.92],[1, 0.55, 0.2, 0.9, 0.88]])
anchors = torch.tensor([[0, 0.1, 0.2, 0.3], [0.15, 0.2, 0.4, 0.4],[0.63, 0.05, 0.88, 0.98], [0.66, 0.45, 0.8, 0.8],[0.57, 0.3, 0.92, 0.9]])

In [53]:
ground_truth[:,1:]

tensor([[0.1000, 0.0800, 0.5200, 0.9200],
        [0.5500, 0.2000, 0.9000, 0.8800]])

In [54]:
box_iou(anchors,ground_truth[:,1:])

inters : tensor([[[0.1000, 0.2000],
         [0.0000, 0.1000]],

        [[0.2500, 0.2000],
         [0.0000, 0.2000]],

        [[0.0000, 0.8400],
         [0.2500, 0.6800]],

        [[0.0000, 0.3500],
         [0.1400, 0.3500]],

        [[0.0000, 0.6000],
         [0.3300, 0.5800]]])
inter_areas:  tensor([[0.0200, 0.0000],
        [0.0500, 0.0000],
        [0.0000, 0.1700],
        [0.0000, 0.0490],
        [0.0000, 0.1914]])


tensor([[0.0536, 0.0000],
        [0.1417, 0.0000],
        [0.0000, 0.5657],
        [0.0000, 0.2059],
        [0.0000, 0.7459]])

In [56]:
torch.max(ground_truth[:,1:],dim=1)

torch.return_types.max(
values=tensor([0.9200, 0.9000]),
indices=tensor([3, 2]))

In [60]:
torch.max(ground_truth[:,1:],ground_truth[:,:-1])

tensor([[0.1000, 0.1000, 0.5200, 0.9200],
        [1.0000, 0.5500, 0.9000, 0.9000]])

In [61]:
torch.max(ground_truth[:,1:],dim=1)

torch.return_types.max(
values=tensor([0.9200, 0.9000]),
indices=tensor([3, 2]))

In [62]:
torch.max(box_iou(anchors,ground_truth[:,1:]),dim=1)

inters : tensor([[[0.1000, 0.2000],
         [0.0000, 0.1000]],

        [[0.2500, 0.2000],
         [0.0000, 0.2000]],

        [[0.0000, 0.8400],
         [0.2500, 0.6800]],

        [[0.0000, 0.3500],
         [0.1400, 0.3500]],

        [[0.0000, 0.6000],
         [0.3300, 0.5800]]])
inter_areas:  tensor([[0.0200, 0.0000],
        [0.0500, 0.0000],
        [0.0000, 0.1700],
        [0.0000, 0.0490],
        [0.0000, 0.1914]])


torch.return_types.max(
values=tensor([0.0536, 0.1417, 0.5657, 0.2059, 0.7459]),
indices=tensor([0, 0, 1, 1, 1]))

In [63]:
ground_truth[:,1:]

tensor([[0.1000, 0.0800, 0.5200, 0.9200],
        [0.5500, 0.2000, 0.9000, 0.8800]])

In [64]:
torch.nonzero(ground_truth[:,1:] >= 0.5).reshape(-1)

tensor([0, 2, 0, 3, 1, 0, 1, 2, 1, 3])

In [66]:
ground_truth[:,1:] 

tensor([[0.1000, 0.0800, 0.5200, 0.9200],
        [0.5500, 0.2000, 0.9000, 0.8800]])

In [65]:
torch.nonzero(ground_truth[:,1:] >= 0.5)

tensor([[0, 2],
        [0, 3],
        [1, 0],
        [1, 2],
        [1, 3]])

In [67]:
max_ious, indices = torch.max(box_iou(anchors,ground_truth[:,1:]),dim=1)

inters : tensor([[[0.1000, 0.2000],
         [0.0000, 0.1000]],

        [[0.2500, 0.2000],
         [0.0000, 0.2000]],

        [[0.0000, 0.8400],
         [0.2500, 0.6800]],

        [[0.0000, 0.3500],
         [0.1400, 0.3500]],

        [[0.0000, 0.6000],
         [0.3300, 0.5800]]])
inter_areas:  tensor([[0.0200, 0.0000],
        [0.0500, 0.0000],
        [0.0000, 0.1700],
        [0.0000, 0.0490],
        [0.0000, 0.1914]])


In [68]:
max_ious

tensor([0.0536, 0.1417, 0.5657, 0.2059, 0.7459])

In [73]:
torch.nonzero(max_ious>=0.5).reshape(-1)

tensor([2, 4])

In [71]:
indices

tensor([0, 0, 1, 1, 1])

In [72]:
indices[max_ious >= 0.5]

tensor([1, 1])

In [74]:
torch.max(box_iou(anchors,ground_truth[:,1:]))

inters : tensor([[[0.1000, 0.2000],
         [0.0000, 0.1000]],

        [[0.2500, 0.2000],
         [0.0000, 0.2000]],

        [[0.0000, 0.8400],
         [0.2500, 0.6800]],

        [[0.0000, 0.3500],
         [0.1400, 0.3500]],

        [[0.0000, 0.6000],
         [0.3300, 0.5800]]])
inter_areas:  tensor([[0.0200, 0.0000],
        [0.0500, 0.0000],
        [0.0000, 0.1700],
        [0.0000, 0.0490],
        [0.0000, 0.1914]])


tensor(0.7459)

In [75]:
gground_truth = ground_truth[:,1:]

In [76]:
num_anchors, num_gt_boxes=anchors.shape[0],gground_truth.shape[0]

In [77]:
num_anchors

5

In [78]:
num_gt_boxes

2

In [80]:
jaccard=box_iou(anchors, gground_truth)

inters : tensor([[[0.1000, 0.2000],
         [0.0000, 0.1000]],

        [[0.2500, 0.2000],
         [0.0000, 0.2000]],

        [[0.0000, 0.8400],
         [0.2500, 0.6800]],

        [[0.0000, 0.3500],
         [0.1400, 0.3500]],

        [[0.0000, 0.6000],
         [0.3300, 0.5800]]])
inter_areas:  tensor([[0.0200, 0.0000],
        [0.0500, 0.0000],
        [0.0000, 0.1700],
        [0.0000, 0.0490],
        [0.0000, 0.1914]])


In [81]:
jaccard

tensor([[0.0536, 0.0000],
        [0.1417, 0.0000],
        [0.0000, 0.5657],
        [0.0000, 0.2059],
        [0.0000, 0.7459]])

In [82]:
anchors_bbox_map = torch.full((num_anchors,), -1)

In [83]:
anchors_bbox_map

tensor([-1, -1, -1, -1, -1])

In [84]:
max_ious, indices = torch.max(jaccard, dim=1)

In [85]:
max_ious

tensor([0.0536, 0.1417, 0.5657, 0.2059, 0.7459])

In [86]:
indices

tensor([0, 0, 1, 1, 1])

In [87]:
anc_i = torch.nonzero(max_ious >= 0.5).reshape(-1)
anc_i

tensor([2, 4])

In [88]:
box_j = indices[max_ious >= 0.5]
box_j

tensor([1, 1])

In [90]:
anchors_bbox_map[anc_i] = box_j
anchors_bbox_map

tensor([-1, -1,  1, -1,  1])

In [91]:
col_discard = torch.full((num_anchors,), -1)
col_discard

tensor([-1, -1, -1, -1, -1])

In [92]:
row_discard = torch.full((num_gt_boxes,), -1)
row_discard

tensor([-1, -1])

In [95]:
jaccard

tensor([[0.0536, 0.0000],
        [0.1417, 0.0000],
        [0.0000, 0.5657],
        [0.0000, 0.2059],
        [0.0000, 0.7459]])

In [97]:
max_idx = torch.argmax(jaccard)

In [99]:
num_gt_boxes

2

In [98]:
 (max_idx % num_gt_boxes).long()

tensor(1)

In [100]:
anc_idx = (max_idx / num_gt_boxes).long()
anc_idx

tensor(4)

In [101]:
anchors_bbox_map

tensor([-1, -1,  1, -1,  1])

In [102]:
jaccard

tensor([[0.0536, 0.0000],
        [0.1417, 0.0000],
        [0.0000, 0.5657],
        [0.0000, 0.2059],
        [0.0000, 0.7459]])

In [108]:
def assign_anchor_to_bbox(groud_truth,anchors,device,iou_threshold=0.5):
    num_anchors,num_gt_boxes=anchors.shape[0],groud_truth.shape[0]
    jaccard=box_iou(anchors,groud_truth)
    anchors_bbox_map=torch.full((num_anchors,),-1,dtype=torch.long,device=device)
    max_ious,indices=torch.max(jaccard,dim=1)
    anc_i=torch.nonzero(max_ious>=0.5).reshape(-1)
    box_j=indices[max_ious>=0.5]
    anchors_bbox_map[anc_i]=box_j
    col_discard=torch.full((num_anchors,),-1)
    row_discard=torch.full((num_gt_boxes,),-1)
    
    for _ in range(num_gt_boxes):
        max_idx=torch.argmax(jaccard)
        box_idx=(max_idx % num_gt_boxes).long()
        anc_idx=(max_idx / num_gt_boxes).long()
        anchors_bbox_map[anc_idx]=box_idx
        jaccard[:,box_idx]=col_discard
        jaccard[anc_idx,:]=row_discard
        
    return anchors_bbox_map    

In [109]:
assign_anchor_to_bbox(gground_truth,anchors,'cpu')

inters : tensor([[[0.1000, 0.2000],
         [0.0000, 0.1000]],

        [[0.2500, 0.2000],
         [0.0000, 0.2000]],

        [[0.0000, 0.8400],
         [0.2500, 0.6800]],

        [[0.0000, 0.3500],
         [0.1400, 0.3500]],

        [[0.0000, 0.6000],
         [0.3300, 0.5800]]])
inter_areas:  tensor([[0.0200, 0.0000],
        [0.0500, 0.0000],
        [0.0000, 0.1700],
        [0.0000, 0.0490],
        [0.0000, 0.1914]])


tensor([-1,  0,  1, -1,  1])

In [111]:
anc_i

tensor([2, 4])

In [110]:
box_j

tensor([1, 1])

In [112]:
jaccard

tensor([[0.0536, 0.0000],
        [0.1417, 0.0000],
        [0.0000, 0.5657],
        [0.0000, 0.2059],
        [0.0000, 0.7459]])

In [114]:
max_ious

tensor([0.0536, 0.1417, 0.5657, 0.2059, 0.7459])

In [113]:
indices

tensor([0, 0, 1, 1, 1])

In [118]:
max_idx

tensor(9)

In [119]:
9%2

1

In [121]:
9/2

4.5

In [122]:
num_gt_boxes

2

In [126]:
(torch.argmax(jaccard) % num_gt_boxes).long()

tensor(1)

In [127]:
(torch.argmax(jaccard) / num_gt_boxes).long()

tensor(4)

In [130]:
(torch.tensor(9) / gground_truth.shape[0]).long()

tensor(4)

In [134]:
torch.Tensor([5.4]).long()

tensor([5])